# Importar Librerías Requeridas
Importa las librerías necesarias, incluyendo pandas para la manipulación de datos y geopy para la geocodificación.

In [1]:
# Importar las librerías necesarias
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from datetime import datetime

# Cargar Datos desde Archivo Parquet
Carga el DataFrame desde el archivo Parquet especificado que contiene la columna 'dirección'.

In [2]:
# Cargar el archivo parquet
file_path = '../../data/processed/inmuebles24_departamentos.parquet'

# Leer el archivo Parquet en un DataFrame
df = pd.read_parquet(file_path)

# Verificar las primeras filas del DataFrame para asegurar que se cargó correctamente
df.head()

,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,cuenta_con_cocina_integral,cuenta_con_sala,cuenta_con_closet,...,cuenta_con_terraza,cuenta_con_comedor,cuenta_con_area_de_lavado,cuenta_con_salon_usos_multiples,cuenta_con_mantenimiento_incluido,cuenta_con_vigilancia_24_horas,direccion,colonia,cp,municipio
0,24000.0,100,2.0,2.0,1.0,0,0,1,1,0,...,0,1,0,0,0,0,san jeronimo lidice san jeronimo lidice la mag...,san jeronimo lidice,10200,la magdalena contreras
1,12000.0,50,1.0,1.0,1.0,1,0,0,1,1,...,0,0,1,0,1,0,callejon del prado barrio san francisco la mag...,el prado,9480,la magdalena contreras
2,34100.0,232,3.0,4.0,2.0,0,0,1,1,1,...,1,1,0,0,0,1,blvd. adolfo ruiz cortines 2775 san jeronimo l...,adolfo ruiz cortines,4630,la magdalena contreras
3,16000.0,165,2.0,1.0,1.0,0,0,0,1,0,...,0,1,0,0,0,0,magnolia 26 san jeronimo lidice la magdalena c...,san jeronimo lidice,10200,la magdalena contreras
4,26000.0,180,3.0,2.0,2.0,0,0,1,0,1,...,0,0,0,0,0,1,san marcos 11 pedregal 2 la magdalena contreras,san marcos,2020,la magdalena contreras


# Inicializar el Geocodificador Nominatim
Crea una instancia del geocodificador Nominatim, especificando un 'user_agent' único para tu aplicación.

In [3]:
# Inicializar el Geocodificador Nominatim
geolocator = Nominatim(user_agent="my_geocoding_app")  # Especificar un 'user_agent' único para la aplicación

# Definir Función para Obtener Coordenadas
Define una función que tome una dirección como entrada, use el geocodificador para encontrar su ubicación y devuelva la latitud y longitud. La función debe manejar casos en los que no se encuentre la dirección.

In [4]:
# Definir la función para obtener coordenadas
def obtener_coordenadas(direccion):
    """
    Toma una dirección como entrada y devuelve la latitud y longitud.
    Si no se encuentra la dirección, devuelve None para ambos valores.
    """
    try:
        # Usar el geocodificador para buscar la dirección
        time.sleep(1)  # Añadir un retraso para respetar las políticas de uso del servicio
        ubicacion = geolocator.geocode(direccion, timeout=10)
        if ubicacion:
            return ubicacion.latitude, ubicacion.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        # Manejar el caso en que el geocodificador exceda el tiempo de espera
        return None, None

# Aplicar Geocodificación a las Direcciones
Aplica la función de geocodificación a la columna 'dirección' del DataFrame para obtener las coordenadas de cada inmueble. Almacena los resultados en nuevas columnas llamadas 'latitud' y 'longitud'.

In [5]:
# Aplicar la función de geocodificación a la columna 'dirección'
df[['latitud', 'longitud']] = df['direccion'].apply(
    lambda direccion: pd.Series(obtener_coordenadas(direccion))
)

# Verificar las primeras filas del DataFrame para confirmar que se agregaron las columnas
df.head()

,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,cuenta_con_cocina_integral,cuenta_con_sala,cuenta_con_closet,...,cuenta_con_area_de_lavado,cuenta_con_salon_usos_multiples,cuenta_con_mantenimiento_incluido,cuenta_con_vigilancia_24_horas,direccion,colonia,cp,municipio,latitud,longitud
0,24000.0,100,2.0,2.0,1.0,0,0,1,1,0,...,0,0,0,0,san jeronimo lidice san jeronimo lidice la mag...,san jeronimo lidice,10200,la magdalena contreras,19.321482,-99.220969
1,12000.0,50,1.0,1.0,1.0,1,0,0,1,1,...,1,0,1,0,callejon del prado barrio san francisco la mag...,el prado,9480,la magdalena contreras,NaN,NaN
2,34100.0,232,3.0,4.0,2.0,0,0,1,1,1,...,0,0,0,1,blvd. adolfo ruiz cortines 2775 san jeronimo l...,adolfo ruiz cortines,4630,la magdalena contreras,19.319711,-99.221621
3,16000.0,165,2.0,1.0,1.0,0,0,0,1,0,...,0,0,0,0,magnolia 26 san jeronimo lidice la magdalena c...,san jeronimo lidice,10200,la magdalena contreras,19.318938,-99.228502
4,26000.0,180,3.0,2.0,2.0,0,0,1,0,1,...,0,0,0,1,san marcos 11 pedregal 2 la magdalena contreras,san marcos,2020,la magdalena contreras,19.307883,-99.228310


# Verificar los Resultados
Muestra las primeras filas del DataFrame actualizado para verificar que las columnas 'latitud' y 'longitud' se han añadido y poblado correctamente.

In [6]:
# Muestra las primeras filas del DataFrame actualizado para verificar las columnas 'latitud' y 'longitud'
df[['direccion', 'latitud', 'longitud']].head()  # Mostrar solo las columnas relevantes para la verificación

,direccion,latitud,longitud
0,san jeronimo lidice san jeronimo lidice la mag...,19.321482,-99.220969
1,callejon del prado barrio san francisco la mag...,NaN,NaN
2,blvd. adolfo ruiz cortines 2775 san jeronimo l...,19.319711,-99.221621
3,magnolia 26 san jeronimo lidice la magdalena c...,19.318938,-99.228502
4,san marcos 11 pedregal 2 la magdalena contreras,19.307883,-99.228310


In [7]:
# Guardar el DataFrame con coordenadas en un nuevo archivo Parquet con la fecha
today_str = datetime.now().strftime('%Y-%m-%d')
output_path = f'../../data/processed/inmuebles24_departamentos_coordenadas_{today_str}.parquet'
df.to_parquet(output_path, index=False)
    

In [8]:
df.shape

(10233, 27)

In [9]:
df.latitud.isnull().sum(), df.longitud.isnull().sum()

(3027, 3027)

In [1]:
3027/10233

0.2958076810319554